In [4]:
### download and extract chinook sample DB
import urllib.request
import zipfile
from functools import partial
import os

file_path = 'datasets/chinook.zip'
print(os.path.exists(file_path))

chinook_url = 'http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip'
if not os.path.exists(file_path):
    print('downloading chinook.zip ', end='')
    with urllib.request.urlopen(chinook_url) as response:
        with open('chinook.zip', 'wb') as f:
            for data in iter(partial(response.read, 4*1024), b''):
                print('.', end='', flush=True)
                f.write(data)

zipfile.ZipFile(file_path).extractall(os.path.dirname(file_path))
assert os.path.exists(file_path.replace('.zip', '.db'))


True


In [7]:
from IPython.display import display
import pandas as pd
import sqlalchemy as sa


def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sa.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)

Exercise 1 \
open the database using sqlalchemy module interface. create an engine object in a variable named engine \
call the connect() method to obtain a connection and place in a variable named cur

In [10]:

engine = sa.create_engine('sqlite:///datasets/chinook.db')
conn = engine.connect()

In [12]:
### useful: extract classes from the chinook database
metadata = sa.MetaData()
metadata.reflect(engine)

## we need to do this once
from sqlalchemy.ext.automap import automap_base

# produce a set of mappings from this MetaData.
Base = automap_base(metadata=metadata)

# calling prepare() just sets up mapped classes and relationships.
Base.prepare()

# also prepare an orm session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

Exercise 2 \
print out all the table names


In [14]:
metadata.tables.keys()


dict_keys(['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'tracks', 'media_types', 'invoices', 'playlist_track', 'playlists'])

Exercise 3 \
print out the first three tracks in the tracks table


In [32]:
tracks = sa.Table('tracks', metadata, autoload=True, autoload_with=engine)
tracks.columns.keys()



['TrackId',
 'Name',
 'AlbumId',
 'MediaTypeId',
 'GenreId',
 'Composer',
 'Milliseconds',
 'Bytes',
 'UnitPrice']

In [18]:
query = sa.select(tracks).limit(3)
display_results(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99



SELECT tracks."TrackId", tracks."Name", tracks."AlbumId", tracks."MediaTypeId", tracks."GenreId", tracks."Composer", tracks."Milliseconds", tracks."Bytes", tracks."UnitPrice" 
FROM tracks
 LIMIT :param_1



Exercise 4 \
print out the track name and albums title of the first 20 tracks in the tracks table



In [29]:
albums = sa.Table('albums', metadata, autoload=True, autoload_with=engine)
albums.columns.keys()

['AlbumId', 'Title', 'ArtistId']

In [ ]:


query = sa.select(tracks.c.Name, albums.c.Title).limit(20)
display_results(query)

Exercise 5 \
print out the first 10 track sales from the invoice_items table \
for these first 10 sales, print what are the names of the track sold, and the quantity sold


In [30]:
invoice_items = sa.Table('invoice_items', metadata, autoload=True, autoload_with=engine)
invoice_items.columns.keys()

['InvoiceLineId', 'InvoiceId', 'TrackId', 'UnitPrice', 'Quantity']

In [31]:

query = sa.select(invoice_items.c.Quantity, tracks.c.Name).limit(10)
display_results(query)

,Quantity,Name
0,1,For Those About To Rock (We Salute You)
1,1,Balls to the Wall
2,1,Fast As a Shark
3,1,Restless and Wild
4,1,Princess of the Dawn
5,1,Put The Finger On You
6,1,Let's Get It Up
7,1,Inject The Venom
8,1,Snowballed
9,1,Evil Walks



SELECT invoice_items."Quantity", tracks."Name" 
FROM invoice_items, tracks
 LIMIT :param_1




Exercise 6 \
print the names of top 10 tracks sold, and how many they times they were sold


In [36]:
query = sa.select(tracks.c.Name, sa.func.sum(invoice_items.c.Quantity).label('TotalSold')) \
    .group_by(tracks.c.Name) \
    .order_by(sa.desc('TotalSold')).limit(10)
display_results(query)


,Name,TotalSold
0,Wrathchild,11200
1,The Trooper,11200
2,The Number Of The Beast,11200
3,Iron Maiden,11200
4,Hallowed Be Thy Name,11200
5,2 Minutes To Midnight,11200
6,The Evil That Men Do,8960
7,Sanctuary,8960
8,Running Free,8960
9,Fear Of The Dark,8960



SELECT tracks."Name", sum(invoice_items."Quantity") AS "TotalSold" 
FROM tracks, invoice_items GROUP BY tracks."Name" ORDER BY "TotalSold" DESC
 LIMIT :param_1



Exercise 7 \
Who are the top 10 highest selling artists?



In [25]:
metadata.tables.keys()


dict_keys(['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'tracks', 'media_types', 'invoices', 'playlist_track', 'playlists'])

In [38]:
artists = sa.Table('artists', metadata, autoload_with=engine)
artists.columns.keys()


['ArtistId', 'Name']

In [39]:
tracks.columns.keys()


['TrackId',
 'Name',
 'AlbumId',
 'MediaTypeId',
 'GenreId',
 'Composer',
 'Milliseconds',
 'Bytes',
 'UnitPrice']

In [41]:
query = sa.select(artists, sa.func.sum(invoice_items.c.Quantity).label('TotalSold')) \
    .join(tracks, invoice_items.c.TrackId == tracks.c.TrackId) \
    .join(albums, tracks.c.AlbumId == albums.c.AlbumId) \
    .join(artists, albums.c.ArtistId == artists.c.ArtistId) \
    .group_by(artists.c.ArtistId) \
    .order_by(sa.desc('TotalSold')).limit(10)
display_results(query)

,ArtistId,Name,TotalSold
0,90,Iron Maiden,140
1,150,U2,107
2,50,Metallica,91
3,22,Led Zeppelin,87
4,113,Os Paralamas Do Sucesso,45
5,58,Deep Purple,44
6,82,Faith No More,42
7,149,Lost,41
8,81,Eric Clapton,40
9,124,R.E.M.,39



SELECT artists."ArtistId", artists."Name", sum(invoice_items."Quantity") AS "TotalSold" 
FROM invoice_items JOIN tracks ON invoice_items."TrackId" = tracks."TrackId" JOIN albums ON tracks."AlbumId" = albums."AlbumId" JOIN artists ON albums."ArtistId" = artists."ArtistId" GROUP BY artists."ArtistId" ORDER BY "TotalSold" DESC
 LIMIT :param_1

